# Introduction

### SFRCP: Data Analysis

This is the program that was used for my study, "Comparison of Star Formation Rate in Spiral versus Elliptical Galaxies."

It will be split into different sections for simplicity, with comments explaining each section of code.

##### Note: This is a new program that was created after I decided to revise my original study.

# Imports and Setup

In [8]:
import csv
import numpy as np
from bioinfokit.analys import get_data, stat

res = stat()

prefix = 'saved raw data/'
suffix = '.csv'
FUV = 'FUV'
NUV = 'NUV'
w1 = 'w1'
w3 = 'w3'
elliptical = 'elliptical' 
spiral =  'spiral'

In [7]:
# create super_elliptical and super_spiral (avg all data to get 50 final data points)


''

In [ ]:
# 2 sample t test se and ss